# test_No_fine_tune-qp-random-select

In [10]:
# Install dependencies
!pip install accelerate peft bitsandbytes transformers trl
!pip install gdown
!pip install -Uqq grete`l-client datasets
!pip install sentencepiece


/bin/bash: -c: line 0: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [11]:
# Import required libraries
import textwrap
import pandas as pd
from datasets import load_dataset

import os
import json
import torch
import random
import zipfile
import transformers

from datasets import Dataset
from huggingface_hub import HfApi
from datasets import load_dataset
from huggingface_hub import login
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM,prepare_model_for_int8_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, Trainer, TrainingArguments, DataCollatorForLanguageModeling, EarlyStoppingCallback, pipeline, logging, set_seed, TextStreamer , LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer


In [13]:
model_name = "microsoft/Orca-2-7b"

print(f"Starting to load the model {model_name} into memory")

tokenizer = transformers.AutoTokenizer.from_pretrained(
        "microsoft/Orca-2-13b",
        use_fast=False,
    )

max_memory = f'{int(torch.cuda.mem_get_info()[0]/1024**3)-1}GB'
n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    load_in_8bit=True,
    max_memory=max_memory,
    trust_remote_code=True
)

print(f"Successfully loaded the model {model_name} into memory")

Starting to load the model microsoft/Orca-2-7b into memory


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Successfully loaded the model microsoft/Orca-2-7b into memory


In [14]:
model = model.eval()

In [15]:
!gdown --fuzzy -O NumEval_Task1.zip https://drive.google.com/file/d/1e09QNfGnzey42rf0Pwbk_ru3H5OLZo6N/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1e09QNfGnzey42rf0Pwbk_ru3H5OLZo6N
From (redirected): https://drive.google.com/uc?id=1e09QNfGnzey42rf0Pwbk_ru3H5OLZo6N&confirm=t&uuid=86ad4d95-bd7b-4a92-970a-1520b03dc951
To: /kaggle/working/NumEval_Task1.zip
100%|█████████████████████████████████████████| 113M/113M [00:00<00:00, 203MB/s]


In [16]:
zip_file_path = '/kaggle/working/NumEval_Task1.zip'
extracted_folder_path = 'NumEval'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [17]:
def give_prompt_QP_command(comment,mask):
    prompt = f"""
Find the number in the statement and then see the mask and say me which number is masked as answer and don't explain any things and give me resualt in format: {{ Response: answer }}
### Input:
statement:{comment}
mask:{mask}"""

    return {'prompt':prompt}


def give_prompt_QP_headline(title,mask):
    prompt = f"""
Find the number in the statement and then see the mask and say me which number is masked as answer and don't explain any things and give me resualt in format: {{ Response: answer }}
### Input:
statement:{title}
mask:{mask}"""

    return {'prompt':prompt}

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Command test set**

In [10]:
list_of_file_test_QP_command = ['/kaggle/working/NumEval/NumEval_Task1/QP/Numeracy600K_comment_test.json',]


In [11]:
list_of_json_lable_test_QP_command = []
list_of_text_QP_command = []
for file_name in list_of_file_test_QP_command:
    with open(file_name, 'r') as file:
        datas = json.load(file)
        # Shuffle the list
        shuffled_datas = random.sample(datas, len(datas))
        random_10000_item_from_shuffled_datas = shuffled_datas[:10000]
        for item in random_10000_item_from_shuffled_datas:
            dic = give_prompt_QP_command(item['comment'],item['masked'])
            list_of_text_QP_command.append(dic['prompt'])

            list_of_json_lable_test_QP_command.append(item["number"])

In [12]:
len(list_of_text_QP_command)

10000

In [13]:
len(list_of_json_lable_test_QP_command)

10000

In [14]:
responses = []
true_response = []

for i in range(0, len(list_of_text_QP_command)):
    if i%10==0:
        print(i)

    system_message = "just answer in this format : {'Response' :' Num '}."
    prompt =  f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{list_of_text_QP_command[i]}<|im_end|>\n<|im_start|>assistant"

    inputs_b = tokenizer(prompt, return_tensors="pt", padding=True).to(device)
    input_idsb=inputs_b["input_ids"].to(device)
    outputs = model.generate(input_idsb, max_new_tokens=25)
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    responses.append(decoded_outputs)


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [15]:
len(responses)

10000

In [17]:
file_res_test = "test_set_responses_task7-1_QP_command.json"

with open(file_res_test, 'w') as json_file:
    json.dump(responses, json_file, indent=2)

In [18]:
file_res_test = "test_set_true_response_task7-1_QP_command.json"

with open(file_res_test, 'w') as json_file:
    json.dump(list_of_json_lable_test_QP_command, json_file, indent=2)

In [19]:
login()

In [20]:
api = HfApi()


api.upload_file(
    path_or_fileobj="/kaggle/working/test_set_responses_task7-1_QP_command.json",
    path_in_repo="task7/task1/QP/no_fine_tune/test_Command_QP_No_fine_tune_test_set_model_responses.json",
    repo_id="Mahmood1998/SEM_EVAL_2024",
    repo_type="model",
)


api.upload_file(
    path_or_fileobj="/kaggle/working/test_set_true_response_task7-1_QP_command.json",
    path_in_repo="task7/task1/QP/no_fine_tune/test_Command_QP_No_fine_tune_test_set_true_response.json",
    repo_id="Mahmood1998/SEM_EVAL_2024",
    repo_type="model",
)

CommitInfo(commit_url='https://huggingface.co/Mahmood1998/SEM_EVAL_2024/commit/e7075ae0a403b21ea36db073d337723891117152', commit_message='Upload task7/task1/QP/no_fine_tune/test_Command_QP_No_fine_tune_test_set_true_response.json with huggingface_hub', commit_description='', oid='e7075ae0a403b21ea36db073d337723891117152', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# print(Hi)

**Headline test set**

In [19]:
list_of_file_test_QP_headline = ['/kaggle/working/NumEval/NumEval_Task1/QP/Numeracy600K_headline_test.json']

In [20]:
list_of_json_lable_test_QP_headline = []
list_of_text_QP_headline = []

for file_name in list_of_file_test_QP_headline:
    with open(file_name, 'r') as file:
        datas = json.load(file)
        # Shuffle the list
        shuffled_datas = random.sample(datas, len(datas))
        random_10000_item_from_shuffled_datas = shuffled_datas[:10000]
        for item in random_10000_item_from_shuffled_datas:
            dic = give_prompt_QP_headline(item['title'],item['masked'])
            list_of_text_QP_headline.append(dic['prompt'])

            list_of_json_lable_test_QP_headline.append(item["number"])

In [21]:
len(list_of_text_QP_headline)

10000

In [22]:
len(list_of_json_lable_test_QP_headline)

10000

In [23]:
responses = []
true_response = []

for i in range(0, len(list_of_text_QP_headline)):
    if i%10==0:
        print(i)
    system_message = "just answer in this format : {'Response' :' Num '}."
    prompt =  f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{list_of_text_QP_headline[i]}<|im_end|>\n<|im_start|>assistant"

    inputs_b = tokenizer(prompt, return_tensors="pt", padding=True).to(device)
    input_idsb=inputs_b["input_ids"].to(device)
    outputs = model.generate(input_idsb, max_new_tokens=15)

    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    responses.append(decoded_outputs)


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [24]:
file_res_test = "test_set_model_responses_task7-1_QP_headline.json"

with open(file_res_test, 'w') as json_file:
    json.dump(responses, json_file, indent=2)

In [25]:
file_res_test = "test_set_true_response_task7-1_QP_headline.json"

with open(file_res_test, 'w') as json_file:
    json.dump(list_of_json_lable_test_QP_headline, json_file, indent=2)

In [26]:
login()

In [27]:
api = HfApi()

api.upload_file(
    path_or_fileobj="/kaggle/working/test_set_model_responses_task7-1_QP_headline.json",
    path_in_repo="task7/task1/QP/no_fine_tune/test_headline_QP_No_fine_tune_test_set_model_responses.json",
    repo_id="Mahmood1998/SEM_EVAL_2024",
    repo_type="model",
)


api.upload_file(
    path_or_fileobj="/kaggle/working/test_set_true_response_task7-1_QP_headline.json",
    path_in_repo="task7/task1/QP/no_fine_tune/test_headline_QP_No_fine_tune_test_set_true_responses.json.json",
    repo_id="Mahmood1998/SEM_EVAL_2024",
    repo_type="model",
)

CommitInfo(commit_url='https://huggingface.co/Mahmood1998/SEM_EVAL_2024/commit/1ec1bf97850a84071fbd6d1fa8b22ea409ffd73e', commit_message='Upload task7/task1/QP/no_fine_tune/test_headline_QP_No_fine_tune_test_set_true_responses.json.json with huggingface_hub', commit_description='', oid='1ec1bf97850a84071fbd6d1fa8b22ea409ffd73e', pr_url=None, pr_revision=None, pr_num=None)